In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np
from torchvision.models import resnet18

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Modify ResNet to fit the small data scenario
def create_small_resnet(num_classes=2, dropout_rate=0.5):
    model = resnet18(pretrained=False)
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(num_features, num_classes)
    )
    return model

In [ ]:
# Load CIFAR-10 data
def load_cifar10_data():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
    return train_dataset

class CustomCIFAR10(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

def sample_task(dataset, num_samples_per_class=5, num_query_per_class=15, used_indices=None):
    if used_indices is None:
        used_indices = set()

    chosen_classes = random.sample(range(10), 2)  # Select two random classes
    class_map = {chosen_classes[i]: i for i in range(2)}
    support_data = []
    query_data = []
    support_labels = []
    query_labels = []
    indices_per_class = {class_map[chosen_classes[0]]: [], class_map[chosen_classes[1]]: []}

    for idx, (image, label) in enumerate(dataset):
        if label in chosen_classes and idx not in used_indices:
            class_label = class_map[label]
            if len(indices_per_class[class_label]) < num_samples_per_class + num_query_per_class:
                if len(indices_per_class[class_label]) < num_samples_per_class:
                    support_data.append(image)
                    support_labels.append(class_label)
                else:
                    query_data.append(image)
                    query_labels.append(class_label)
                indices_per_class[class_label].append(idx)
                used_indices.add(idx)
            if all(len(indices_per_class[c]) == num_samples_per_class + num_query_per_class for c in indices_per_class):
                break

    return CustomCIFAR10(support_data, support_labels), CustomCIFAR10(query_data, query_labels)


In [ ]:
def euclidean_dist(x, y):
    return torch.cdist(x, y)

def prototypical_loss(support_features, query_features, support_labels, query_labels):
    # Calculate prototypes as the mean of support features by class
    unique_labels = torch.unique(support_labels)
    prototypes = torch.stack([support_features[support_labels == label].mean(0) for label in unique_labels])

    # Calculate distances from query features to prototypes
    dists = euclidean_dist(query_features, prototypes)

    # Use log_softmax and negative log likelihood loss
    log_p_y = torch.nn.functional.log_softmax(-dists, dim=1)
    loss_val = torch.nn.functional.nll_loss(log_p_y, query_labels)
    return loss_val



def meta_train(model, dataset, device, epochs=50, tasks_per_epoch=5, num_support=5, num_query=15):
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        used_indices = set()
        total_loss = 0
        for _ in range(tasks_per_epoch):
            support_set, query_set = sample_task(dataset, num_samples_per_class=num_support, num_query_per_class=num_query, used_indices=used_indices)
            support_loader = DataLoader(support_set, batch_size=len(support_set), shuffle=True)
            query_loader = DataLoader(query_set, batch_size=len(query_set), shuffle=True)

            support_data, support_labels = next(iter(support_loader))
            query_data, query_labels = next(iter(query_loader))

            support_data, support_labels = support_data.to(device), support_labels.to(device)
            query_data, query_labels = query_data.to(device), query_labels.to(device)

            optimizer.zero_grad()
            support_features = model(support_data)
            query_features = model(query_data)
            loss = prototypical_loss(support_features, query_features, support_labels, query_labels)  # Updated to remove the extra argument
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch {epoch + 1}, Average Loss: {total_loss / tasks_per_epoch}')

# Initialize datasets and model
train_dataset = load_cifar10_data()
model = create_small_resnet().to(device)

# Start training
meta_train(model, train_dataset, device)

Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1, Average Loss: 0.7401832818984986
Epoch 2, Average Loss: 0.6789878845214844
Epoch 3, Average Loss: 0.933695811033249
Epoch 4, Average Loss: 0.6123790979385376
Epoch 5, Average Loss: 0.6614694952964782
Epoch 6, Average Loss: 0.8506425738334655
Epoch 7, Average Loss: 0.7290533542633056
Epoch 8, Average Loss: 0.7865317404270172
Epoch 9, Average Loss: 0.7868565201759339
Epoch 10, Average Loss: 0.7292192101478576
Epoch 11, Average Loss: 0.5491008162498474
Epoch 12, Average Loss: 0.7582954883575439
Epoch 13, Average Loss: 0.683092612028122
Epoch 14, Average Loss: 0.4206447660923004
Epoch 15, Average Loss: 0.542730736732483
Epoch 16, Average Loss: 0.7506886333227157
Epoch 17, Average Loss: 0.6584505200386047
Epoch 18, Average Loss: 0.6465480774641037
Epoch 19, Average Loss: 0.5795291006565094
Epoch 20, Average Loss: 0.5400517821311951
Epoch 21, Average Loss: 0.7901002883911132
Epoch 22, Average Loss: 0.410729306936264
Epoch 23, Average Loss: 0.5999743461608886
Epoch 24, Average Loss: 

In [ ]:
def meta_test(model, dataset, device, num_samples_per_class=5, num_support=3, num_query=2, num_tasks=10):
    accuracies = []
    model.eval()  # Set the model to evaluation mode
    used_indices = set()  # Initialize here to track indices across tasks

    with torch.no_grad():
        for _ in range(num_tasks):
            # Sample a new task
            support_set, query_set = sample_task(dataset, num_samples_per_class=num_samples_per_class, num_query_per_class=num_query, used_indices=used_indices)
            support_loader = DataLoader(support_set, batch_size=len(support_set), shuffle=True)
            query_loader = DataLoader(query_set, batch_size=len(query_set), shuffle=False)

            # Get data for support and query sets
            support_data, support_labels = next(iter(support_loader))
            query_data, query_labels = next(iter(query_loader))

            support_data, support_labels = support_data.to(device), support_labels.to(device)
            query_data, query_labels = query_data.to(device), query_labels.to(device)

            # Calculate prototypes
            support_features = model(support_data)
            unique_labels = torch.unique(support_labels)
            prototypes = torch.stack([support_features[support_labels == label].mean(0) for label in unique_labels])

            # Evaluate on query data
            query_features = model(query_data)
            dists = euclidean_dist(query_features, prototypes)
            _, predicted = torch.min(dists, 1)

            # Calculate accuracy
            correct = (predicted == query_labels).sum().item()
            total = query_labels.size(0)
            accuracy = correct / total
            accuracies.append(accuracy)

    average_accuracy = np.mean(accuracies)
    print(f'Average Test Accuracy on new tasks: {average_accuracy * 100:.2f}%')
    return average_accuracy


# Prepare a test dataset (assuming classes not seen during training)
def load_cifar10_test_data():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    return test_dataset


# Load test dataset
test_dataset = load_cifar10_test_data()

# Evaluate the model on new tasks
meta_test(model, test_dataset, device, num_samples_per_class=5, num_support=3, num_query=2, num_tasks=5)


Files already downloaded and verified
Average Test Accuracy on new tasks: 70.00%


0.7